# Figure 2 : Ripple oscillations

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from neuropy import plotting
from neuropy.utils import signal_process
import subjects
import pandas as pd
from scipy import stats

sd_sessions = subjects.sd.allsess[:-1]
nsd_sessions = subjects.nsd.allsess

print(sd_sessions)
print(nsd_sessions)

## Ripple band power during SD, RS and NSD

## Calculating ripple rate during sleep deprivation and recovery sleep

In [ ]:
sd_rpl_rate_df = pd.DataFrame()
rs_rpl_rate_df = pd.DataFrame()
nsd_rpl_rate_df = pd.DataFrame()
for sub, sess in enumerate(sd_sessions):
    post = sess.paradigm["post"]
    sd_period = [post[0], post[0] + 5 * 3600]
    rpls = sess.ripple.starts

    binsz = 900

    n_bins = 5 * 3600 / binsz
    sd_bin = np.arange(n_bins + 1) * binsz + sd_period[0]
    hist_rpl = np.histogram(rpls, bins=sd_bin)[0]
    hist_rpl = hist_rpl / np.sum(hist_rpl)

    sd_rpl_rate_df  =sd_rpl_rate_df.append(
        pd.DataFrame(
            {
                "n_ripples": hist_rpl / binsz,
                "tbin": (sd_bin[:-1] - sd_bin[0]) / 3600,
                "sub": sub,
            }
        ),
        ignore_index=True,
    )

    rs_period = [post[0]+5*3600,post[1]]
    n_bins =  np.diff(rs_period)/ binsz
    rs_bin = np.arange(n_bins + 1) * binsz + rs_period[0]
    hist_rpl = np.histogram(rpls, bins=rs_bin)[0]
    hist_rpl = hist_rpl / np.sum(hist_rpl)

    rs_rpl_rate_df  =rs_rpl_rate_df.append(
        pd.DataFrame(
            {
                "n_ripples": hist_rpl / binsz,
                "tbin": (rs_bin[:-1] - rs_bin[0]) / 3600,
                "sub": sub,
            }
        ),
        ignore_index=True,
    )


for sub, sess in enumerate(nsd_sessions):
    post = sess.paradigm["post"]
    rpls = sess.ripple.starts
    binsz = 900
    n_bins = np.diff(post)/ binsz
    post_bin = np.arange(n_bins + 1) * binsz +post[0]
    hist_rpl = np.histogram(rpls, bins=post_bin)[0]
    hist_rpl = hist_rpl / np.sum(hist_rpl)

    nsd_rpl_rate_df  =nsd_rpl_rate_df.append(
        pd.DataFrame(
            {
                "n_ripples": hist_rpl / binsz,
                "tbin": (post_bin[:-1] - post_bin[0]) / 3600,
                "sub": sub,
            }
        ),
        ignore_index=True,
    )



## Plotting

In [ ]:
%matplotlib widget
from neuropy import plotting

figure = plotting.Fig()
fig,gs= figure.draw(grid=(4,3))
fig.suptitle('Figure 2')

#--- panel a --------

rpl_indx = [112,6050]
rpl_chan = 89
sd_sess = sd_sessions[5]
rpl_df = sd_sess.ripple.to_dataframe()
rpl_colors = ['#6ce953','#d3bc0d']

# gs_filt = figure.subplot2grid(gs[0,:2],grid=(2,1))
# for i,(rpl_ind,rpl_color) in enumerate(zip(rpl_indx,rpl_colors)):
#     rpl_start,rpl_stop =rpl_df.iloc[rpl_ind].start,rpl_df.iloc[rpl_ind].stop
#     rpl_peaktime = rpl_df.iloc[rpl_ind].peaktime
#     ax = plt.subplot(gs_filt[i])
#     signal = sd_sess.eegfile.get_signal(channel_indx=rpl_chan,t_start=rpl_start-3,t_stop=rpl_start+3)
#     filt_signal = signal_process.filter_sig.bandpass(signal=signal,lf=150,hf=250)
#     plotting.plot_signal_traces(filt_signal, ax=ax,lw=0.5,color='gray')
#     ax.scatter(rpl_peaktime,1.2,s=20,marker='v',c=rpl_color)

#     if i==0:
#         # ax.text(0,0.5,'Early')
#         ax.set_title('Ripple band (150-250 Hz)')
#         figure.panel_label(ax=ax,label='A',fontsize=14)


gs_rpl = figure.subplot2grid(gs[0,0],grid=(1,2))
for i,(rpl_ind,rpl_color) in enumerate(zip(rpl_indx,rpl_colors)):
    ax_rpl = plt.subplot(gs_rpl[i])
    rpl_start,rpl_stop =rpl_df.iloc[rpl_ind].start,rpl_df.iloc[rpl_ind].stop
    rpl_chans = sd_sess.probegroup.get_connected_channels(groupby='shank')[5]
    rpl_starts = sd_sess.ripple.starts
    rpl_stops = sd_sess.ripple.stops
    rpl = sd_sess.eegfile.get_signal(channel_indx=rpl_chans.astype('int'),t_start=rpl_start-0.05,t_stop = rpl_stop+0.02)
    plotting.plot_signal_traces(rpl,ax=ax_rpl,pad=0.8,color=rpl_color)
    ax_rpl.set_yticklabels('')
    # if i==0:
    # ax_rpl.set_title(f't = {}')


#----- panel b,c,d ---------
rpl_colors = [subjects.sd_colors['sd'],subjects.sd_colors['sd'],subjects.sd_colors['nsd']]
dfs = [sd_rpl_rate_df,rs_rpl_rate_df,nsd_rpl_rate_df]
for i,(data,col) in enumerate(zip(dfs,rpl_colors)): 
    ax_rate1 = plt.subplot(gs[1,i])
    sns.lineplot(data=data,x='tbin',y='n_ripples',ci='sd',ax=ax_rate1,color=col)
    ax_rate1.set_title('Sleep deprivation')
    ax_rate1.set_xlabel('ZT time (h)')
    ax_rate1.set_ylabel('Norm. ripple rate')
    ax_rate1.ticklabel_format(axis='y',style='sci',scilimits=(0,0))
    figure.panel_label(ax_rate1,'B',fontsize=14)

#----- panel e (Figure caption) ---------
ax_rate4 = plt.subplot(gs[2,:])
ax_rate4.text(0,0.5,'Figure 2: Increasing rate of ripples during sleep deprivation. \n(A) Ripple band local field potential. Example ripples on the right. \n(B) Ripple rate during sleep deprivation and recovery sleep of SD session.\n(C) Same as A but during NSD session',wrap=True)
ax_rate4.axis('off')


figure.savefig(subjects.figpath_sd/'figure2')